In [62]:
import requests
from bs4 import BeautifulSoup as bs
import re
import smtplib
from email.mime.text import MIMEText

In [42]:
def crawl_swai():
    urls = []
    for i in range(5):
        endpoint = 'https://swai.smu.ac.kr/bbs/board.php?bo_table=07_01&page=' + str(i + 1)
        soup = bs(requests.get(endpoint).text, 'html.parser')

        tableText = soup.find('div', {'class' : 'tbl_head01 tbl_wrap'}).find('tbody')
        articles = [a['href'] for a in tableText.findAll('a')]
        urls.extend(articles)

    res = [url.split('=')[-2].split('&')[0] for url in urls]
    return res

In [47]:
def no_space(text):
  text1 = re.sub('\nbsp;|&nbsp;|\n|\t|\r|  ', '', text)
  text2 = re.sub('\n\n', '', text1)
  return text2

In [48]:
def get_content_swai(id):
    endpoint = 'https://swai.smu.ac.kr/bbs/board.php?bo_table=07_01&wr_id=' + str(id)
    soup = bs(requests.get(endpoint).text, 'html.parser')
    article = soup.find('article', {'id' : 'bo_v'})
    
    title = article.find('span', {'class' : 'bo_v_tit'}).text
    title = no_space(title).strip()
    date = ' '.join(article.find('strong', {'class' : 'if_date'}).text.split(' ')[1:])
    content = article.find('div', {'id' : 'bo_v_con'})
    res = {
        'url' : endpoint,
        'title' : title,
        'date' : date,
        'content' : content
    }
    return res

In [66]:
def config_mail(content):
    html = '<html>' + '<header><h1><a href = ' + str(content['url']) + '>' + str(content['title']) + '<a></h1><heeader>' + '<h4>' + str(content['date']) + '</h4>' + str(content['content']) + '</html>'
    return html

In [67]:
def send_mail(targets, content):
    email = 'kritiasMailSender@gmail.com'
    password = 'ufayshdyupbwqilj'

    session = smtplib.SMTP('smtp.gmail.com', 587)
    session.starttls()
    session.login(email, password)

    msg = MIMEText(content, 'html')
    msg['Subject'] = '오늘의 사업단 글입니다.'
    for target in targets:
        session.sendmail("Secretary", target, msg.as_string())

In [89]:
def add_ids(ids):
    f = open('data.txt', 'r')
    datum = f.readline().split(' ')
    for id in ids:
        if id not in datum:
            datum.append(id)
    datum = ' '.join(datum)
    f.close()
    f = open('data.txt', 'w')
    f.write(datum)
    f.close()

In [88]:
def check_ids(ids):
    datum = []
    res = []
    with open('data.txt', 'r') as f:
        datum = f.readline().split(' ')
    for id in ids:
        if id not in datum:
            res.append(id)
    return res

In [94]:
ids = ['123', '124', '125']
#add_ids(ids)
check_ids(ids)

[]

In [95]:
def set_all_read():
    ids = crawl_swai()
    add_ids(ids)

In [96]:
set_all_read()

In [97]:
targets = ['smree@naver.com']
ids = crawl_swai()
ids = check_ids(ids)
add_ids(ids)
for id in ids:
    content = get_content_swai(ids)
    mail_body = config_mail(content)
    send_mail(targets, mail_body)

[]
